In [1]:
from pymongo import MongoClient

client = MongoClient('192.168.9.169', 27017)

db = client.Fb_Raw

location = db.Fb_Location

In [2]:
def remove_accents(input_str):
    s1 = u'ÀÁÂÃÈÉÊÌÍÒÓÔÕÙÚÝàáâãèéêìíòóôõùúýĂăĐđĨĩŨũƠơƯưẠạẢảẤấẦầẨẩẪẫẬậẮắẰằẲẳẴẵẶặẸẹẺẻẼẽẾếỀềỂểỄễỆệỈỉỊịỌọỎỏỐốỒồỔổỖỗỘộỚớỜờỞởỠỡỢợỤụỦủỨứỪừỬửỮữỰựỲỳỴỵỶỷỸỹ'
    s0 = u'AAAAEEEIIOOOOUUYaaaaeeeiioooouuyAaDdIiUuOoUuAaAaAaAaAaAaAaAaAaAaAaAaEeEeEeEeEeEeEeEeIiIiOoOoOoOoOoOoOoOoOoOoOoOoUuUuUuUuUuUuUuYyYyYyYy'
    s = ''
    for c in input_str:
        if c in s1:
            s += s0[s1.index(c)]
        else:
            s += c
    return s

In [3]:
def clean(string):
    newItem = remove_accents(string)
    newItem = newItem.lower()
    newItem = newItem.replace(" ", "")
    return newItem

In [4]:
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)
    if len(s2) == 0:
        return len(s1)

    s1 = clean(s1)
    s2 = clean(s2)
    
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1 
            deletions = current_row[j] + 1       
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row

    return previous_row[-1]

In [5]:
hihi = 0
class Node:
    nodeId = 0
    parent = None
    children = []
    items = []
    
    def setParent(self, node):
        self.parent = node
        
    def setChildren(self, nodes):
        self.children = nodes
        
    def addItem(self, node):
        self.items.append(node)
        
    def setItem(self, nodes):
        self.items = nodes
        
    def __str__(self):
        st = "\n" + str(self.nodeId) + ": " + str(self.items) + "\n" + "Children of " + str(self.items[0]) + ": " 
        if (len(self.children) > 0):
            for c in self.children:
                st += str(c) + "\n"
        return st
    
    def toDict(self):
        itsDict = dict()
        children = []
        for c in self.children:
            children.append(c.toDict())
        itsDict[list(self.items)[0]] = children
        return itsDict
    
    def __init__(self):
        global hihi
        self.nodeId = hihi
        self.items = []
        self.parent = None
        self.children = []
        hihi +=1
    
    def numOfCharacter(self):
        sum = 0
        for i in self.items:
            sum += len(i)
        return sum
    
    def distance(self, item):
        sum = 0
        for i in self.items:
            sum += levenshtein(i, item)
        return sum/self.numOfCharacter()
    
    def satisfy(self, word, threshold):
        if word is None:
            return False
        if remove_accents(word) in remove_accents(str(self.items)):
            return True
        elif self.distance(word) < threshold:
            return True
        return False
    
    def satisfy2(self, word, threshold):
        for n in self.children:
            if n.satisfy(word, threshold):
                return n

In [6]:
def getNumberOfElement(listGroup):
    sum = 0
    for group in listGroup:
        sum += len(group.items)
    return sum

import random
def getOne(available):
    value = random.uniform(0.0, 1.0)
    n = getNumberOfElement(available)
    total = 0
    for a in available:
        total += len(a.items)/n
        if (total >= value):
            return a.nodeId
        
def findNodeById(nodes, nodeId):
    for i in range(len(nodes)):
        if nodes[i].nodeId == nodeId:
            return i

In [7]:
# chance = True --> find the children of node
def fit(g, value, threshold, chance = False):
    input_str = value.get("name").replace("[", "").replace("]", "")
    s = input_str.split(", ")
    s = s[::-1]
    
    available = []
    for n in g:
        if (not chance) and n.satisfy(s[0], threshold):
            available.append(n)
        elif chance and n.satisfy2(s[0], threshold):
            available.append(n)
    if len(available) == 1:
        if not chance:
            available[0].addItem(s[0])
            if len(s) > 1:
                s.remove(s[0])
                s = s[::-1]
                value["name"] = str(s).replace("\'", "")
                available[0].children = fit(available[0].children, value, threshold)
        else:
            available[0] = fit(available[0].children, value, threshold)
    elif len(available) == 0:
        check = True
        for group in g:
            if len(group.children) > 0:
                for c in group.children:
                    if c.satisfy2(s[0], threshold):
                        check = False
                        group.children = fit(group.children, value, threshold, True)
                        break
            if not check:
                return g
        if check:
            if chance:
                nodes = []
                for i in range(len(s)):
                    node = Node()
                    node.addItem(s[i])
                    if i != 0:
                        node.parent = nodes[i-1]
                        nodes[i-1].children.append(node)
                    nodes.append(node) 
                g.append(nodes[0])
            else:
                g = fit(g, value, threshold, True)

    elif len(available) > 1:
        a = findNodeById(g, getOne(available))
        g[a].addItem(s[0])
#     todo check children of available
                
    return g

In [8]:
def classify(data, num, target):
    inputData = []
    if num < 0:
        return target
    for d in data:
        if d["name"].count(",") == num:
            target.append(d)
        else:
            inputData.append(d)
    classify(inputData, num-1, target)
    return target

In [16]:
g = []
hihi = 0
threshold = 0.2
import time
start = time.time()
dtd = location.find().limit(1000)
data = []
dt = []
for d in dtd:
    data.append(d)
    
dt = classify(data, 3, dt)

for d in dt:
    g = fit(g, d, threshold)
    
end = time.time()
print(end-start)

50.02311301231384


In [19]:
import json
for group in g:
    group = group.toDict()
    group = str(group).replace("{\'", "{\"").replace("\':", "\":")
    group = group.replace("\"\"" , "\"" )
    group = json.loads(group)
    print(json.dumps(group, indent=2, sort_keys=True, ensure_ascii = False))
#     print(group)

{
  "Vietnam": [
    {
      "Tuyên Quang": [
        {
          "Tuyên Quang": []
        }
      ]
    },
    {
      "Nam Định": [
        {
          "Nam Định": []
        }
      ]
    },
    {
      "Thái Bình": [
        {
          "Thái Bình": []
        },
        {
          "Hung Nhan": []
        },
        {
          "Dong Dong": []
        },
        {
          "Quynh Coi": []
        }
      ]
    },
    {
      "Kiến Giang": [
        {
          "Vàm Rầy (1)": []
        },
        {
          "Long An": [
            {
              "Long Hoa": []
            },
            {
              "Can Gioc": []
            },
            {
              "Binh Hiep": []
            },
            {
              "Tân An": []
            },
            {
              "Thanh Hoa": []
            }
          ]
        },
        {
          "Mitho": []
        },
        {
          "Châu Thành": []
        },
        {
          "Vĩnh Bình (1)": []
        },
        {
  